# Pre-training Strategies 

The BERT model is pre-trained on the following two tasks:

- Masked language modeling 
- Next sentence prediction 

Let us understand how the above two pre-training strategies work by looking at them one by one. Before directly diving into the masked language modeling task, first, let us understand how a language modeling task works. 


## Language Modeling

In the language modeling task, we train the model to predict the next word given a sequence of words. We can categorize the language modeling into two:

- Auto-regressive language modeling
- Auto-encoding language modeling 

### Auto-regressive language modeling 

We can categorize the auto-regressive language modeling into:

- Forward (left-to-right) prediction
- Backward (right-to-left) prediction

Let's understand how these two methods work with an example. Consider a text 'Paris is a beautiful city. I love Paris'. Let's remove the word the 'city' and add blank as shown below:

Paris is a beautiful __.  I love Paris

Now, our model has to predict the blank. If we use the forward prediction, then our model reads all the words from left to right till the blank to make a prediction as shown below:

Paris is a beautiful __

If we use backward prediction, then our model reads all the words from right to left till the blank to make a prediction as shown below:

__. I love Paris

Thus, auto-regressive models are unidirectional in nature meaning that they read the sentence in only one direction. 

### Auto-encoding language modeling
The auto-encoding language modeling takes advantage of both forward (left-to-right) and backward (right-to-left) prediction. That is, it reads the sentence in both directions while making a prediction. Thus, we can say that the auto-encoding language model is bidirectional in nature. As we can observe from below, to predict the bank, the auto-encoding language model reads the sentence in both directions, that is, left-to-right and right-to-left:

Paris is a beautiful __. I love Paris

The bidirectional model gives better results because if we read the sentence from both directions, it will give us more clarity on understanding the sentence. 

Now that we have understood how language modeling works, in the next section, we will look into one of the pre-training strategies of BERT called masked language modeling. 

### Masked Language Modelling

BERT is an autoencoding language model meaning that it reads the sentence in both directions to make a prediction. In masked language modeling task, in a given input sentence, we randomly mask 15% of the words and train the network to predict the masked words. To predict the masked words, our model reads the sentence in both directions and tries to predict the masked words.

Let's understand how masked language modeling works with an example. Let's take the same sentences we saw earlier: 'Paris is a beautiful city', 'I love Paris'. First, we tokenize the sentences and get the tokens as shown below:

tokens = [Paris, is, a beautiful, city, I, love, Paris]

Now, we add [CLS] token at the beginning of the first sentence and [SEP] token at the end of every sentence as shown below:

tokens = [ [CLS], Paris, is, a beautiful, city, [SEP], I, love, Paris, [SEP] ]

Next, we randomly mask 15% of the tokens (words) from our token list. Say, we mask the word city, then we replace the word city with a [MASK] token as shown below:

tokens = [ [CLS], Paris, is, a beautiful, [MASK], [SEP], I, love, Paris, [SEP] ]

As we can observe from the preceding tokens list, we have replaced the word city with [MASK] token. Now we train our BERT model to predict the masked token.

There is a small catch here. Masking tokens in this way will create a discrepancy between pre-training and finetuning. That is, we learned that we train the BERT by predicting the [MASK] token. After training, we can finetune the pre-trained BERT for downstream tasks say sentiment analysis. But during finetuning, we will not have any [MASK] tokens in the input. So it will cause a mismatch between the way how BERT is pre-trained and how it is used for finetuning. 

To overcome this issue, we apply the 80-10-10% rule. We learned that we randomly mask 15% of the tokens in the sentence, now, out of these 15% tokens, we do the following:

- For 80% of the time, we replace the token (actual word) with the [MASK] token. So for 80% of the time, the input to the model will be:
tokens = [ [CLS], Paris, is, a beautiful, [MASK], [SEP], I, love, Paris, [SEP] ]

- For 10% of the time, we replace the token (actual word) with a random token (random word). So for 10% of the time, the input to the model will be:
tokens = [ [CLS], Paris, is, a beautiful, love, [SEP], I, love, Paris, [SEP] ]

- For 10% of the time, we don't make any changes. So for 10% of the time, the input to the model will be:
tokens = [ [CLS], Paris, is, a beautiful, city, [SEP], I, love, Paris, [SEP] ]

After tokenization and masking, we feed the input tokens to the token, segment, and position embedding layers and get the input embeddings.

Next, we feed the input embedding to the BERT. As shown in the following figure, BERT takes the input and returns the representation of each token as an output. $R_{\text{[CLS]}}$ denotes the representation of the token [CLS], $R_{\text{Paris}}$ denotes the representation of the token Paris, and so on. In this example, we use the BERT-base which has 12 encoder layers,  12 attention heads, 768 hidden units. Since we use the BERT-base model, the size of the representation of each token will be 768:



![title](images/13.png)

From the preceding figure, we observe that we obtained the representation  of each of the tokens. Now, how to predict the masked token with these representations? 

To predict the masked token, we feed the representation of the masked token $R_{\text{[MASK]}}$ returned by the BERT to the feedforward network with a softmax activation. Now, the feedforward network takes the $R_{\text{[MASK]}}$ as input and returns the probability of all the words in our vocabulary to be the masked word as shown in the following figure. In the following figure, input embedding layers (token, segment, and position) are not shown to reduce the clutter:



![title](images/14.png)

As shown in the preceding figure, we have a high probability for the word 'city' to be the masked word. So, in this case, the masked word will be predicted as 'city'. 

Note that in the initial iterations, our model will not return the correct probability because the weights of the feedforward network and encoder layers of BERT will not be optimal. But over a series of iterations, with backpropagation, we update the weights of the feedforward network and encoder layers of the BERT and learn the optimal weights. 

The masked language modeling task is also known as cloze task. We learned how the masked language modeling task works and how we train the BERT model using the masked language modeling task. While masking the input tokens, we can also use a slightly different method called whole word masking. We will learn whole word masking in the next section. 

## Whole word masking 
Let's understand how the Whole Word Masking (WWM) works with an example. Consider the sentence, "Let us start pretraining the model". Remember that BERT uses WordPiece tokenizer, so, after tokenizing the sentence using the WordPiece tokenizer, we will have the following tokens:

tokens = [let, us, start, pre, ##train, ##ing, the, model]

Next, we add [CLS] token at the beginning of the sentence and [SEP] token at the end of the sentence: 

tokens = [ [CLS], let, us, start, pre, ##train, ##ing, the, model, [SEP] ]

Now, we randomly mask 15% of words. Suppose, we have the following:

tokens = [ [CLS], [MASK], us, start, pre, [MASK], ##ing, the, model, [SEP] ]

As we can observe from above, we have masked the words 'let' and '##train'. Note that the word ##train is actually a subword and it is a part of the word pretraining. In the whole word masking method, if the subword is masked then we mask all the words corresponding to that subword. Thus, now our tokens become: 

tokens = [ [CLS], [MASK], us, start, [MASK], [MASK], [MASK], the, model, [SEP] ]

As we can observe from above, all the tokens corresponding to subword ##train is masked. Thus, in the whole word masking, if a subword is masked then we mask all the words corresponding to that subword. Note that we also have to retain our mask rate which is 15%. So, while masking all the words corresponding to subwords, we can ignore masking other words if it exceeds 15%. As shown in the following, we have ignored masking the word 'let' to retain the mask rate: 

tokens = [ [CLS], let, us, start, [MASK], [MASK], [MASK], the, model, [SEP] ]

Thus, in this way, we mask tokens using whole word masking. After masking, we feed the input tokens to BERT and train the model to predict the masked token just like we learned in the previous section. 

Now that we have learned how to train the BERT using a masked language modeling task, in the next section, we look into another interesting task for training BERT. 

## Next sentence Prediction 
Next sentence prediction (NSP) is another interesting strategy used for training the BERT model. Next sentence prediction is a binary classification task. In the NSP task, we feed two sentences to the BERT and it has to predict whether the second sentence is the follow-up (next sentence) of the first sentence. Let's understand the next sentence prediction task with an example.

Consider the following two sentences: 

Sentence A: She cooked pasta. 

Sentence B: It was delicious.

In the above sentence pair, sentence B is a follow-up, that is, the next sentence of sentence A. So, we label this sentence pair as isNext indicating that sentence B is the next sentence of sentence A. 

Now, consider the following two sentences: ￼￼

Sentence A: Turn the radio on.

Sentence B: She bought a new hat. 

In the above sentence pair, sentence B is not a follow-up, that is, not the next sentence of sentence A. So, we label this sentence pair as notNext indicating that sentence B is not the next sentence of sentence A. 

In the next sentence prediction task, the goal of our model is to predict whether the sentence pair belongs to isNext or notNext category. We feed the sentence pair (sentence A and B) to the BERT and train it to predict whether sentence B is the next sentence of sentence A. The model returns isNext if sentence B is the next sentence of sentence A else it will return notNext as an output. Thus, the next sentence prediction is essentially a binary classification task. 

But what is the use of the next sentence prediction task? By performing the next sentence prediction task, our model can understand the relation between the two sentences. Understanding the relation between two sentences is useful for many downstream tasks such as question answering, text generation, and so on.  

Okay, how can we obtain the dataset for the NSP task? We can generate the dataset from any monolingual corpus. Say, we have a couple of documents. For isNext class, we take any two consecutive sentences from one document and label them as isNext and for the notNext class, we take one sentence from one document and another sentence from a random document and label them as notNext.Note that we need to maintain 50% of data points in the isNext class and 50% of data points in the notNext class.

Now that we learned what the NSP task is, let's see how to train the BERT model for performing NSP task. Suppose, our dataset looks like as shown in the following:



![title](images/15.png)


Let's take the first data point in the preceding sample dataset. First, we will tokenize the sentence pair as shown below:

tokens = [She, cooked, pasta, It, was, delicious]

Next, we add [CLS] token only at the beginning of the first sentence and [SEP] token at the end of every sentence as shown below:

tokens = [[CLS], She, cooked, pasta, [SEP], It, was, delicious, [SEP]]

Now, we feed the input tokens to the token, segment, and position embedding layers and get the input embeddings. Then, we feed the input embeddings to the BERT and obtain the representation of each token. As shown in the following figure,  $R_{\text{[CLS]}}$ denotes the representation of the token [CLS], $R_{\text{she}}$ denotes the representation of the token she, and so on: 


![title](images/16.png)

We learned that NSP is a binary classification task. But now we have only the representation of each token in the sentence pair. How can we classify the sentence pair based on these representations? 

To perform classification, we simply take the representation of the [CLS] token and feed it to the feedforward network with softmax function which then returns the probability of our sentence pair being isNext and NotNext. Wait! Why do we need to take the embedding of [CLS] token alone? Why not the embedding of other tokens? 

The [CLS] token basically holds the aggregate representation of all the tokens. So, it basically holds the aggregate representation of our sentences. Thus, we can ignore the representation of all other tokens and simply take the representation of the [CLS] token $R_{CLS}$ and feed it to the feedforward layer with a softmax function which returns the probability. This is shown in the following figure. Note that in the below figure, the input embedding layers (token, segment, and position embedding layer) are not shown to reduce the clutter: 


![title](images/17.png)

From the preceding figure, we can understand that the feedforward network returns the high probability of our input sentence pair to be in the isNext class. 

Note that in the initial iterations, our model will not return the correct probability because the weights of the feedforward network and encoder layers of BERT will not be optimal. But over a series of iterations, with backpropagation, we update the weights of the feedforward network and encoder layers of the BERT and learn the optimal weights. 

In this way, we train our BERT model with the next sentence prediction task. We learned how to pre-train the BERT using masked language modeling and next sentence prediction tasks. In the next section, we will look into the pre-training procedure. 



